In [75]:
! pip install indic-nlp-library --quiet
! pip install pyiwn --quiet

In [76]:
import pyiwn 
import random
import nltk
import pandas as pd
from indicnlp.tokenize import indic_tokenize
iwn = pyiwn.IndoWordNet(lang=pyiwn.Language.HINDI) 

In [77]:
def get_synonyms(word):
    #Get synonyms of word in the specified language.
    synonyms=[]
    try:
      all_synsets = iwn.synsets(word)
    except:
      return []
    for synset in all_synsets:
        lemmas = synset.lemma_names()
        for lemma in lemmas:
            if lemma != word:
                synonyms.append(lemma)
    return synonyms

In [78]:
def random_insertion(text, n=3):
    #Randomly insert n synonymous words into text.
    tokens = indic_tokenize.trivial_tokenize(text)
    for _ in range(n):
        random_index = random.randint(0, len(tokens)-1)
        random_word = tokens[random_index]
        synonyms = get_synonyms(random_word)
        # print(synonyms)
        if len(synonyms) > 1:
            random_synonym = random.choice(synonyms)
            while random_synonym == random_word:
                random_synonym = random.choice(synonyms)
            tokens[random_index] = random_synonym
    return ' '.join(tokens)

In [79]:
# random_insertion("लगभग 1300 हेक्टेयर ज़मीन का अधिग्रहण किया जा",random.randint(0,len(sentence)))

In [80]:
# Read input CSV file as DataFrame
df = pd.read_csv('/content/drive/MyDrive/NLP/data/clean-hindi-train.csv')
df = df[df.text.notnull()]
df['text'] = df['text'].str[:100]
df.head()

<ipython-input-80-ebf44d45b398>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str[:100]


,label,text
0,1,मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से...
1,1,प्रतीक खुलेपन का आज़ाद ख्याली का और भीड़ से अल...
2,1,ख़ासकर पिछले 10 साल तक प्रधानमंत्री रहे मनमोहन...
3,1,चीनी भाषा में ‘नीहाव’ (क्या हाल हैं) कहकर वो अ...
4,1,मुंबई पुलिस की क्राईम ब्रांच को इस जांच का जिम...


In [81]:
dfaug = df
dfaug['text'] = dfaug['text'].apply(lambda x: random_insertion(x, random.randint(0,len(x))))

In [82]:
# Write updated DataFrame to output CSV file
df = df.append(dfaug, ignore_index=True)
df.to_csv('RandomInsertionAugData.csv', index=False)

<ipython-input-82-cc70630bbcb6>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfaug, ignore_index=True)
